In [1]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
# 📌 Import Seaborn for sample datasets
import seaborn as sns

# 📌 Load the built-in "tips" dataset
# This dataset contains restaurant bill information — total bill, tip,
# gender, smoker/non-smoker, day of the week, time of day, and party size.
#
# We will later use this dataset to demonstrate **Support Vector Regression (SVR)**,
# predicting continuous values (e.g., predicting the TOTAL BILL amount from the Other Idependent features).
tips = sns.load_dataset('tips')

# 📌 Display the first 5 rows of the dataset
# This helps us understand the structure and columns we will work with.
display(tips.head())


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
# 📌 Display dataset structure, categorical distribution, and missing values

# 1️⃣ Check overall dataset information
#    - Number of rows & columns
#    - Column names & data types
#    - Count of non-null values (for missing value detection)
print("🔍 Dataset Info:\n" + "-"*60)
print(tips.info())


# 2️⃣ Check distribution of categorical column "sex"
#    Helps us understand if dataset is balanced (Male vs Female)
print("\n👥 Gender Distribution (sex column):\n" + "-"*60)
print(tips['sex'].value_counts(), "\n")


# 3️⃣ Check for missing values in every column
#    .isnull().sum() → shows how many nulls exist per column
print("❗ Missing Values per Column:\n" + "-"*60)
print(tips.isnull().sum())

🔍 Dataset Info:
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB
None

👥 Gender Distribution (sex column):
------------------------------------------------------------
sex
Male      157
Female     87
Name: count, dtype: int64 

❗ Missing Values per Column:
------------------------------------------------------------
total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64


In [4]:
# 📌 Identify all categorical columns (object + category type)
categorical_cols = tips.select_dtypes(include=['category']).columns

print("🔎 Categorical Column Value Counts")
print("=" * 60)

# Loop through each categorical column and display counts
for col in categorical_cols:
    print(f"\n📌 Column: {col}")
    print("-" * 60)
    print(tips[col].value_counts())


🔎 Categorical Column Value Counts

📌 Column: sex
------------------------------------------------------------
sex
Male      157
Female     87
Name: count, dtype: int64

📌 Column: smoker
------------------------------------------------------------
smoker
No     151
Yes     93
Name: count, dtype: int64

📌 Column: day
------------------------------------------------------------
day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

📌 Column: time
------------------------------------------------------------
time
Dinner    176
Lunch      68
Name: count, dtype: int64


In [5]:
tips.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [6]:
## Independent and dependent features

# 📌 Selecting the INPUT features (X)
# We are using the following columns to PREDICT the total restaurant bill:
#   tip     → numeric (how much tip was given)
#   sex     → categorical (gender of customer)
#   smoker  → categorical (smoker / non-smoker)
#   day     → categorical (day of the week)
#   time    → categorical (Lunch / Dinner)
#   size    → numeric (number of people at the table)
#
# Note: Categorical columns will later need to be encoded to numeric values
#       because Support Vector Regression (SVR) cannot interpret string labels.
X = tips[['tip', 'sex', 'smoker', 'day', 'time', 'size']]


# 📌 Selecting the TARGET variable (y)
# We want the model to PREDICT → total_bill
#   total_bill is a continuous number → ✔ suitable for REGRESSION
y = tips.total_bill


In [7]:
# 📌 Splitting the dataset into Training and Testing sets
from sklearn.model_selection import train_test_split

# train_test_split divides the dataset into two parts:
#   🔹 Training Set → used to TRAIN the model (learn patterns)
#   🔹 Test Set → used to EVALUATE how well the model performs on unseen data
#
# Parameters used:
#   X  → input features (tip, sex, smoker, day, time, size)
#   y  → target variable (total_bill)
#   test_size = 0.25 → 25% data for testing, 75% for training
#                     (dataset is small, so 75/25 split gives a healthy balance)
#   random_state = 10 → ensures the SAME split every time the code runs
#                       (important for reproducibility)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=10
)

In [8]:
# 📌 Feature Encoding (Label Encoding for selected categorical variables)
from sklearn.preprocessing import LabelEncoder

# We will encode the categorical columns "sex", "smoker", and "time" using Label Encoding.
# The "day" column is intentionally NOT label-encoded here, because it will be handled later
# using **One-Hot Encoding**, which is a better choice when category order does NOT matter.

# Create separate label encoders for each categorical column
sex_label = LabelEncoder()
smoker_label = LabelEncoder()
time_label = LabelEncoder()

# ⚠ IMPORTANT ML RULE
# Always FIT encoders on TRAINING DATA ONLY, then use the SAME encoder to transform TEST DATA.
# This avoids data leakage and keeps numeric mappings consistent.

# Encode 'sex'
X_train['sex'] = sex_label.fit_transform(X_train['sex'])
X_test['sex']  = sex_label.transform(X_test['sex'])

# Encode 'smoker'
X_train['smoker'] = smoker_label.fit_transform(X_train['smoker'])
X_test['smoker']  = smoker_label.transform(X_test['smoker'])

# Encode 'time'
X_train['time'] = time_label.fit_transform(X_train['time'])
X_test['time']  = time_label.transform(X_test['time'])

# 📌 NOTE ABOUT "day" COLUMN
# The "day" column remains unencoded at this step on purpose.
# We will apply **One-Hot Encoding** later because:
#   🔹 'day' has no natural order (Thu, Fri, Sat, Sun)
#   🔹 One-Hot Encoding prevents the model from wrongly assuming ranking between days
#
# Example (later): Thu → [1,0,0,0], Fri → [0,1,0,0], Sat → [0,0,1,0], Sun → [0,0,0,1]

# Display encoded training dataset
display(X_train.head())


,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2


In [9]:
## 📌 One-Hot Encoding using ColumnTransformer

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


# ColumnTransformer allows us to apply transformations to **specific columns**
# while leaving the rest of the columns untouched.
#
# Here we apply One-Hot Encoding ONLY to the **'day' column**.
# The 'day' column contains non-numeric categories: Thu, Fri, Sat, Sun
# Since these categories have NO natural order, One-Hot Encoding is the best option.
#
# 🔹 OneHotEncoder(drop='first') → drops the first category to avoid the **dummy variable trap**
#     Example Encoding:
#       Thu → [0,0,0]      (dropped)
#       Fri → [1,0,0]
#       Sat → [0,1,0]
#       Sun → [0,0,1]
#
# [3] → refers to the column index position of 'day' inside X_train
#       (0=tip, 1=sex, 2=smoker, 3=day, 4=time, 5=size)
#
# remainder='passthrough' → keep all remaining columns as they are
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), [3])   # index 3 = 'day'
    ],
    remainder='passthrough'
)


# ⚠ IMPORTANT
# We **fit** the transformer ONLY on TRAINING DATA
# so it learns the list of categories from training set only.
# Then we **transform** both TRAIN and TEST data using the same mapping.
X_train = column_transformer.fit_transform(X_train)   # learn + transform
X_test  = column_transformer.transform(X_test)         # only transform


# Now X_train and X_test contain:
#  🔹 new One-Hot encoded columns for "day"
#  🔹 AND all original numeric + label-encoded columns (passed through unchanged)


In [10]:
## 📌 Applying Support Vector Regression (SVR)

from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 📌 Step 1 — Create the SVR model
# SVR (Support Vector Regression) predicts continuous values instead of class labels.
# Default kernel = 'rbf', which is suitable for most non-linear regression problems.
svr = SVR()    # You may tune hyperparameters later (C, epsilon, gamma)


# 📌 Step 2 — Train (fit) the SVR model
# The model learns the mathematical relationship between:
#   X_train → independent features (tip, sex, smoker, day encoded, time, size)
#   y_train → target value (total_bill)
svr.fit(X_train, y_train)


# 📌 Step 3 — Make predictions on TEST data
# SVR now predicts total_bill values for unseen observations in X_test
y_pred = svr.predict(X_test)


In [11]:
# 📌 Calculate Regression Evaluation Metrics

# 1️⃣ Mean Absolute Error (MAE)
# Average of absolute differences → "on average, how much are we off?"
mae = mean_absolute_error(y_test, y_pred)

# 2️⃣ Mean Squared Error (MSE)
# Penalizes larger errors more strongly (squares the error)
mse = mean_squared_error(y_test, y_pred)

# 3️⃣ Root Mean Squared Error (RMSE)
# Square root of MSE → interpretable on the same scale as the target variable
rmse = np.sqrt(mse)

# 4️⃣ R² Score (Coefficient of Determination)
# Measures how well the model explains variations in target values
#   1.0 → Perfect model
#   0.0 → No better than guessing average
# < 0  → Worse than the baseline model
r2 = r2_score(y_test, y_pred)

# 📌 Print results clearly
print("📊 SVR Regression Metrics")
print("=" * 40)
print(f"🔹 Mean Absolute Error (MAE): {mae:.4f}")
print(f"🔹 Mean Squared Error (MSE): {mse:.4f}")
print(f"🔹 Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"🔹 R² Score: {r2:.4f}")


📊 SVR Regression Metrics
🔹 Mean Absolute Error (MAE): 4.1486
🔹 Mean Squared Error (MSE): 45.6921
🔹 Root Mean Squared Error (RMSE): 6.7596
🔹 R² Score: 0.4603


In [12]:
import pandas as pd 

# 📌 Method to predict the total bill for NEW / UNSEEN customer data
# This function takes raw input values (same features used during training),
# applies the SAME preprocessing steps (label encoding + one-hot encoding),
# and then uses the trained SVR model to predict the total_bill.

def predict_bill(tip, sex, smoker, day, time, size):

    # 🔹 Create a 1-row DataFrame from the input values
    #   — replicates the same column structure the model was trained on
    sample = pd.DataFrame([[tip, sex, smoker, day, time, size]],
                          columns=['tip', 'sex', 'smoker', 'day', 'time', 'size'])

    # 🔹 Apply the SAME label encoders used during model training
    #     (encoders must NEVER be refit — always transform only)
    sample['sex']    = sex_label.transform(sample['sex'])
    sample['smoker'] = smoker_label.transform(sample['smoker'])
    sample['time']   = time_label.transform(sample['time'])

    # 🔹 Apply the SAME one-hot encoder (ColumnTransformer) used during training
    #     — ensures unseen data has the exact same feature structure as training data
    sample = column_transformer.transform(sample)

    # 🔹 Predict using the trained Support Vector Regression model
    predicted_value = svr.predict(sample)[0]   

    # Return the numeric prediction (converted to Python float)
    return float(predicted_value)


In [13]:
total_bill = predict_bill(
    tip=5.00,
    sex='Male',
    smoker='No',
    day='Sun',
    time='Dinner',
    size=3
)

print(f"→ Predicted total_bill: ${total_bill:.2f} dollars")


→ Predicted total_bill: $26.27 dollars


In [14]:
## 📌 Hyperparameter Tuning using GridSearchCV for SVR

from sklearn.model_selection import GridSearchCV

# 📌 Parameter grid for Support Vector Regression (SVR)
# We tune:
#   🔹 C      → Regularization strength (how much the model tolerates error)
#   🔹 gamma  → Controls the curvature / influence of individual training points
#   🔹 kernel → Type of decision function to learn
#
# Kernels included:
#   'rbf'   → Best for most non-linear regression patterns (default & most commonly used)
#   'poly'  → Useful when the relationship between features and target is polynomial in nature
#   'sigmoid' → Behaves like a neural network activation function; works well in some datasets
#
# ⚠ 'linear' kernel is NOT included here because:
#    → total_bill has a non-linear dependency on tip, day, time, smoker etc.
#    → earlier results proved non-linear kernels perform better for this dataset

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],                  # Higher C = more focus on fitting training data
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],        # Higher gamma = more curvature / complexity
    'kernel': ['rbf', 'poly', 'sigmoid']          # Choosing kernels suitable for non-linear regression
}

# 📌 Initialize GridSearchCV
# GridSearchCV will train multiple SVR models (one for each parameter combination)
# and select the BEST MODEL based on cross-validation performance.
grid = GridSearchCV(
    estimator=SVR(),
    param_grid=param_grid,
    scoring='r2',        # R² score chosen to measure regression quality
    cv=5,                # 5-fold cross-validation for more reliable evaluation
    verbose=3,           # Show training progress
    n_jobs=-1,           # Use all CPU cores for faster search
      refit=True         # Retrain the best model on the FULL training dataset
)


In [15]:
# 📌 Train GridSearchCV on the training data
# This step performs the full hyperparameter search:
#   🔹 Trains multiple SVR models — one for EVERY combination in param_grid
#   🔹 Evaluates each model using 5-fold cross-validation (cv=5)
#   🔹 Selects the BEST hyperparameter combination based on scoring='r2'
#   🔹 Because refit=True → it automatically RETRAINS the final SVR model
#       using the BEST hyperparameters on the ENTIRE training dataset.
#
# ⏳ This step may take time because dozens of models are trained internally.
grid.fit(X_train, y_train)


Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.058 total time=   0.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.067 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.113 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.145 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.089 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.328 total time=   0.0s
[CV 1/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.083 total time=   0.0s
[CV 2/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.074 total time=   0.0s
[CV 3/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.174 total time=   0.0s
[CV 4/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.013 total time=   0.0s
[CV 5/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.111 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf

,estimator,SVR()
,param_grid,"{'C': [0.1, 1, ...], 'gamma': [1, 0.1, ...], 'kernel': ['rbf', 'poly', ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,5
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,kernel,'sigmoid'


In [16]:
# 📌 Beautiful printing of best parameters from GridSearchCV

print("\n🏆 Best Hyperparameters Found:")
print("-" * 40)
for key, value in grid.best_params_.items():
    print(f"🔹 {key:<10} → {value}")
print("-" * 40)



🏆 Best Hyperparameters Found:
----------------------------------------
🔹 C          → 1000
🔹 gamma      → 0.001
🔹 kernel     → sigmoid
----------------------------------------
